Convert the Keras model and weights to a TensorFlow SavedModel '.pb' file.

In [1]:
%matplotlib inline

from pathlib import Path
import zipfile
import tempfile
import yaml

import tensorflow as tf
import numpy as np

import sys
sys.path.append("..")
import mask_lib as mlib

root_dir = Path("/home/hadim/.data/Neural_Network/Mask-RCNN/Microtubules/")

model_dir = root_dir / "logs"
model_dir.mkdir(parents=True, exist_ok=True)

saved_model_dir = root_dir / "saved_model"
saved_model_dir.mkdir(parents=True, exist_ok=True)

model_name = "coco_512_new"

/home/hadim/local/conda/envs/nn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Import Model 

In [2]:
config = mlib.MicrotubuleInferenceConfig()
config.set_images_per_gpu(1)

model = mlib.load_model(model_dir, config, mode="inference")
mlib.load_weights(model, init_with="last", model_name=model_name)

# Export model to Tensoflow .pb file

In [3]:
tf_model_path = saved_model_dir / f"tf_model_{model_name}.pb"
zip_tf_model_path = saved_model_dir / f"{tf_model_path.stem}.zip"

In [4]:
mlib.export_to_tensorflow(model, tf_model_path, tf_model_zip_path=zip_tf_model_path)
tf_model_path

INFO:tensorflow:Froze 384 variables.
Converted 384 variables to const ops.


PosixPath('/home/hadim/.data/Neural_Network/Mask-RCNN/Microtubules/saved_model/tf_model_coco_512_new.pb')

# Add pre- and post-processing models to the ZIP file

Both models have been created with this notebook: [`5_Build_Processing_Graph.ipynb`](5_Build_Processing_Graph.ipynb)

In [5]:
preprocessing_model_path = saved_model_dir / "preprocessing_graph.pb"
postprocessing_model_path = saved_model_dir / "postprocessing_graph.pb"

with zipfile.ZipFile(zip_tf_model_path, "a") as z:
    z.write(preprocessing_model_path, arcname=preprocessing_model_path.name)
    z.write(postprocessing_model_path, arcname=postprocessing_model_path.name)

# Add parameters to the ZIP file

In [6]:
parameters = {}
parameters["model_name"] = config.NAME
parameters["class_names"] = config.CLASS_NAMES
parameters["image_min_dimension"] = config.IMAGE_MIN_DIM
parameters["image_max_dimension"] = config.IMAGE_MAX_DIM
parameters["minimum_scale"] = config.IMAGE_MIN_SCALE
parameters["mean_pixels"] = config.MEAN_PIXEL.tolist()
parameters["class_ids"] = np.zeros(len(config.CLASS_NAMES)).tolist()
parameters["backbone_strides"] = list(config.BACKBONE_STRIDES)
parameters["rpn_anchor_scales"] = list(config.RPN_ANCHOR_SCALES)
parameters["rpn_anchor_ratios"] = list(config.RPN_ANCHOR_RATIOS)
parameters["rpn_anchor_stride"] = config.RPN_ANCHOR_STRIDE
    
_, temp_path = tempfile.mkstemp()

with open(temp_path, "w") as f:
    f.write(yaml.dump(parameters))
    
with zipfile.ZipFile(zip_tf_model_path, "a") as z:
    z.write(temp_path, arcname="parameters.yml")
    
parameters

{'model_name': 'microtubule',
 'class_names': ['BG', 'microtubule'],
 'image_min_dimension': 100,
 'image_max_dimension': 512,
 'minimum_scale': 1.0,
 'mean_pixels': [43.53, 39.56, 48.22],
 'class_ids': [0.0, 0.0],
 'backbone_strides': [4, 8, 16, 32, 64],
 'rpn_anchor_scales': [8, 16, 32, 64, 128],
 'rpn_anchor_ratios': [0.5, 1, 2],
 'rpn_anchor_stride': 1}